# Создание новых признаков

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import kstest

__Для начала, ещё раз посмотрим на то, как выглядит наша таблица, по которой мы будем пробовать создавать новые признаки__

In [407]:
df = pd.read_csv('chist_df_whith_loc.csv')
df.head(3)

,YEAR,DISORDER_TYPE,EVENT_TYPE,SUB_EVENT_TYPE,ACTOR1,INTER1,ACTOR2,INTER2,INTERACTION,CIVILIAN_TARGETING,COUNTRY,LOCATION,LATITUDE,LONGITUDE,NOTES,FATALITIES,index,classification
0,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,2,Civilians (Algeria),7,27,Civilian targeting,Algeria,Douaouda,36.672,2.789,5 January: Beheading of 5 citizens in Douaouda...,5,0.624,Lower middle income
1,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,2,Civilians (Algeria),7,27,Civilian targeting,Algeria,Hassasna,36.133,0.883,Two citizens were beheaded in Hassasna.,2,0.624,Lower middle income
2,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,2,Civilians (Algeria),7,27,Civilian targeting,Algeria,Hassi El Abed,34.966,-0.290,Two citizens were killed in a raid on the vill...,2,0.624,Lower middle income


__Добавим сначала простой бинарный признак__, принимающий значение $0$, если конфликт был мирный (Protests, Strategic developments) и $1$, если конфликт был насильственный (Violence against civilians, Battles, Explosions/Remote violence, Protests)

In [123]:
def binar(df):
    df['bool'] = np.ones(np.shape(df)[0])
    df['bool'] = np.where(((df['EVENT_TYPE']== 'Protests') | (df['EVENT_TYPE']== 'Strategic developments')), 0, 1)
    return df

In [408]:
binar(df).head(3)

,YEAR,DISORDER_TYPE,EVENT_TYPE,SUB_EVENT_TYPE,ACTOR1,INTER1,ACTOR2,INTER2,INTERACTION,CIVILIAN_TARGETING,COUNTRY,LOCATION,LATITUDE,LONGITUDE,NOTES,FATALITIES,index,classification,bool
0,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,2,Civilians (Algeria),7,27,Civilian targeting,Algeria,Douaouda,36.672,2.789,5 January: Beheading of 5 citizens in Douaouda...,5,0.624,Lower middle income,1
1,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,2,Civilians (Algeria),7,27,Civilian targeting,Algeria,Hassasna,36.133,0.883,Two citizens were beheaded in Hassasna.,2,0.624,Lower middle income,1
2,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,2,Civilians (Algeria),7,27,Civilian targeting,Algeria,Hassi El Abed,34.966,-0.290,Two citizens were killed in a raid on the vill...,2,0.624,Lower middle income,1


__Теперь мы хотим придумать шкалу, которая будет оценивать уровень насилия. Для этого нам надо разобраться в специфике данных.__

Пытаясь оценить уровень и тяжесть насилия, мы будем обращать внимание на признак SUB_EVENT_TYPE -  это информация о самом событии. Очевидно, что если оно менее сопряжено с кровью и убийствами, наш индекс должен быть меньше. Другим важным признаком является INTERACTION, который говорит нам о том, какие конкретно агенты вступали в противостояние. Нам кажется, если противостояние наблюдалось между организованными военными группами на территории страны, то это сопряжено с большим насилием нежели, например, взаимодействие протестующих и полиции. Конечно, огромное значение ишрает признак FATALITIES, однако для некоторых событий количество жертв просто не установлено, поэтому нельзя полностью на него полагаться.

Конечно, значение этого индекса все еще будет субъективным, однако мы постараемся, чтобы он отражал наше видение проблемы насилия, а также был полезен в дальнейшем. 


Некоторые пояснения мы будем считать событее менее насильственным, если оно вызвано столкновением неорганизованных груп, так, например, MILITARY VERSUS CIVILIANS (государственные репрессии против гражданских лиц) будут считаться чуть более насильственным событием, чем POLITICAL MILITIA VERSUS CIVILIANS (репрессии, осуществляемые проправительственными военизированными формированиями; нападения на гражданских лиц со стороны политических военизированных формирований)

Также мы будем считать военное столкновение с внешней страной более насильственной, чем некоторые виды взаимодействия внутри страны

__Поработаем с  INTERACTION__

#### Для начала, разметим данные и для более удобного восприятия поместим их в таблицу

In [410]:
st = '10 -  дистанционное насилие с участием государственных военных без зарегистрированных жертв, ненасильственные военные операции\n90 - сражения между военными и мятежными войсками\n85 - например, насилие в гражданской войне между государственными силами и повстанцами\n80 -  насилие между полицией и военизированными формированиями политических партий\n60 - например, военное взаимодействие с общинным ополчением\n49 - например, подавление вооуруженной демонстрации полицией или военными\n30 - например, подавление невооруженной демонстрации полицией или военными\n91 - например, государственные репрессии против гражданских лиц\n83 - например, межгосударственный конфликт; взаимодействие государства с частными силами безопасности или операция ООН\n46 - например, создание базы повстанцев; насилие с участием повстанческих групп без заявленной цели; \n92 - насилие между повстанцами\n87 - например, насилие в ходе гражданской войны между повстанцами и проправительственным ополчением; \n80 - например, насилие между повстанцами и местными службами безопасности\n65 - например, спонтанное  насилие против группы повстанцев; насильственная демонстрация с участием группы повстанцев\n69 - например, насилие против протестующих со стороны повстанцев\n95 - например, нападение повстанцев на гражданских лиц\n89 - например, насилие в гражданской войне между повстанцами и военными союзного государства; насилие повстанцев против операции ООН\n23 - стратегический поджог в качестве запугивания со стороны политической партии\n30 - например, межэлитное насилие\n42 - насилие между политическим ополчением и местными службами безопасности\n36 - например, насильственная демонстрация против политической партии; спонтанное насилие против политической партии\n12 - например, мирная демонстрация с участием политической партии\n58 - например, государственные репрессии, осуществляемые проправительственными военизированными формированиями; нападения на гражданских лиц со стороны политических военизированных формирований\n86 - например, насилие между проправительственными ополченцами и внешними государственными военными силами\n24 - например, уничтожение имущества местной милицией\n34 - например, межобщинное насилие\n29 - насильственная демонстрация против местного ополчения\n8 - мирная демонстрация, в которой участвует местная милиция\n39 - например, нападения на гражданских лиц со стороны местной полиции\n59 - например, внешние государственные военные применяют насилие против общинного ополчения\n19 - например, односторонняя насильственная демонстрация; спонтанный поджог\n33 - например, двусторонняя демонстрация с применением насилия, в которой обе стороны применяют насилие\n27 - например, двухсторонняя демонстрация, в которой только одна сторона применяет насилие\n57 - насильственная демонстрация, в ходе которой гражданские лица получают ранения - 34 -  например, насильственная демонстрация, в ходе которой ранены/убиты гражданские лица; спонтанное насилие, когда гражданские лица становятся мишенью толпы\n36 - например, насилие толпы против региональных или международных групп\n1 - например, односторонний мирный протест\n3 - например, двухсторонний мирный протест\n14 - например, мирные протестующие вступают в бой с гражданскими лицами\n9 - например, мирная демонстрация с привлечением частных сил безопасности\n1 - мирные столкн\n21 - ин.гос между собой\n62 - частные силы безопасности против гражданских лиц\n13 - дистанционное насилие со стороны внешних военных сил без заявленной цели; ненасильственные внешние военные операции\n13 - столкновения ин.агентов'
ll = st.split('\n')
ind = []
trext = []
for i in range(len(ll)):
    one = ll[i].split(' - ')
    ind.append(float(one[0]))
    trext.append(one[-1])

In [411]:
index = [10,11,12,13,14,15,16,17,18,20,22,23,24,25,26,27,28,30,33,34,35,36,37,38,40,44,45,46,47,48,50,55,56,57,58,60,66,67, 68,70,77,78, 80, 88]
ras = ['SOLE MILITARY ACTION',
       'MILITARY VERSUS MILITARY',
       'MILITARY VERSUS REBELS',
       'MILITARY VERSUS POLITICAL MILITIA',
       'MILITARY VERSUS COMMUNAL MILITIA',
       'MILITARY VERSUS RIOTERS',
       'MILITARY VERSUS PROTESTERS',
       'MILITARY VERSUS CIVILIANS',
       'MILITARY VERSUS OTHER',
       'SOLE REBEL ACTION',
       'REBELS VERSUS REBELS', 
       'REBELS VERSUS POLITICAL MILIITA',
       'REBELS VERSUS COMMUNAL MILITIA',
       'REBELS VERSUS RIOTERS',
       'REBELS VERSUS PROTESTERS',
       'REBELS VERSUS CIVILIANS',
       'REBELS VERSUS OTHERS',
       'SOLE POLITICAL MILITIA ACTION',
       'POLITICAL MILITIA VERSUS POLITICAL MILITIA',
       'POLITICAL MILITIA VERSUS COMMUNAL MILITIA',
       'POLITICAL MILITIA VERSUS RIOTERS',
       'POLITICAL MILITIA VERSUS PROTESTERS',
       'POLITICAL MILITIA VERSUS CIVILIANS',
       'POLITICAL MILITIA VERSUS OTHERS',
       'SOLE COMMUNAL MILITIA ACTION',
       'COMMUNAL MILITIA VERSUS COMMUNAL MILITIA',
       'COMMUNAL MILITIA VERSUS RIOTERS',
       'COMMUNAL MILITIA VERSUS PROTESTERS',
       'COMMUNAL MILITIA VERSUS CIVILIANS',
       'COMMUNAL MILITIA VERSUS OTHER',
       'SOLE RIOTER ACTION',
       'RIOTERS VERSUS RIOTERS',
       'RIOTERS VERSUS PROTESTERS',
       'RIOTERS VERSUS CIVILIANS',
       'RIOTERS VERSUS OTHERS',
       'SOLE PROTESTER ACTION',
       'PROTESTERS VERSUS PROTESTERS', 
       'PROTESTERS VERSUS CIVILIANS',
       'PROTESTERS VERSUS OTHER',
       'SOLE OTHER ACTOR',
       'OTHER VERSUS OTHER',
       'OTHER ACTOR VERSUS CIVILIANS',
       'SOLE OTHER ACTION', 
       'CIVILIANS VERSUS CIVILIANS']

coef = ind 
texts = trext

table1 = pd.DataFrame({'index': index ,'расшифровка': ras, 'coef': ind, 'Пояснение': trext})
table1 = table1.set_index('index')



In [412]:
table1.head()

,расшифровка,coef,Пояснение
index,,,
10,SOLE MILITARY ACTION,10.0,дистанционное насилие с участием государствен...
11,MILITARY VERSUS MILITARY,90.0,сражения между военными и мятежными войсками
12,MILITARY VERSUS REBELS,85.0,"например, насилие в гражданской войне между го..."
13,MILITARY VERSUS POLITICAL MILITIA,80.0,насилие между полицией и военизированными фор...
14,MILITARY VERSUS COMMUNAL MILITIA,60.0,"например, военное взаимодействие с общинным оп..."
15,MILITARY VERSUS RIOTERS,49.0,"например, подавление вооуруженной демонстрации..."
16,MILITARY VERSUS PROTESTERS,30.0,"например, подавление невооруженной демонстраци..."
17,MILITARY VERSUS CIVILIANS,91.0,"например, государственные репрессии против гра..."
18,MILITARY VERSUS OTHER,83.0,"например, межгосударственный конфликт; взаимод..."


#### Итак, немного о содержании колонок:
   - расшифровка - в ней содержится информация о типе Interaction
   - coef - присвоенный коэффициент степени насилия для конкретного типа взаимодействия
   - пояснение - небольшое словесное описание того, что конкретно из себя представляет тот или иной Interaction

__Проделаем теперь то же самое, но уже с признаком Sub_event_type__

In [416]:
st2 = 'Suicide bomb\n68_Excessive force against protesters\n21_Arrests\n32_Attack\n81_Armed clash\n95_Remote explosive/landmine/IED\n91_Shelling/artillery/missile attack\n91_Peaceful protest\n1_Change to group/activity\n24_Looting/property destruction\n31_Government regains territory\n90_Disrupted weapons use\n100_Protest with intervention\n6_Violent demonstration\n26_Mob violence\n35_Air/drone strike\n87_Other\n1Arrests\n19Excessive force against protesters\n12_Abduction/forced disappearance\n64Suicide bomb\n72_Grenade\n43_Non-violent transfer of territory\n4_Headquarters or base established\n37_Agreement\n1_Non-state actor overtakes territory\n87_Sexual violence\n85_Chemical weapon\n100'
ll2 = st2.split('_')
ind2 = []
name = []
for i in range(len(ll2)):
    one = ll2[i].split('\n')
    ind2.append(float(one[-1]))
    name.append(one[0])




In [417]:
table2 = pd.DataFrame({'type': name, 'coef': ind2})
table2 = table2.set_index('type')
table2.head()

,coef
type,
Suicide bomb,68.0
Excessive force against protesters,21.0
Arrests,32.0
Attack,81.0
Armed clash,95.0
Remote explosive/landmine/IED,91.0
Shelling/artillery/missile attack,91.0
Peaceful protest,1.0
Change to group/activity,24.0


#### Аналогично,о содержании колонки:
   - coef - присвоенный коэффициент степени насилия для конкретного типа вспомогательного события

__Теперь мы предлагаем формулу, которая будет определять степень насилия для конфликтов__:

$$violence\_ ind = \sqrt[3]{interaction \cdot sub \_ event \_ type \cdot (fatalities +1)}$$

#### В соответствие с ней, посчитаем значение violence_ind для каждого события в нашей таблице и объединим с начальными данными

In [398]:
def v_ind(df, table1, table2):
    coef_l = []
    for i in range(len(df)):
        ite_ind = df['INTERACTION'][i]
        ite = table1.loc[ite_ind]['coef']
        typ_ind = df['c'][i]
        typ = table2.loc[typ_ind]['coef']
        deth = df['FATALITIES'][i]
        coef = np.round(((ite*typ*(deth+1)))**(1/3), 2)
        coef_l.append(coef)
    df['violence_ind']= coef_l
    return coef_l

In [399]:
xx = v_ind(df, table1, table2)

In [400]:
df['violence_ind']=xx

In [419]:
df.head(3)

,YEAR,DISORDER_TYPE,EVENT_TYPE,SUB_EVENT_TYPE,ACTOR1,INTER1,ACTOR2,INTER2,INTERACTION,CIVILIAN_TARGETING,COUNTRY,LOCATION,LATITUDE,LONGITUDE,NOTES,FATALITIES,index,classification,bool
0,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,2,Civilians (Algeria),7,27,Civilian targeting,Algeria,Douaouda,36.672,2.789,5 January: Beheading of 5 citizens in Douaouda...,5,0.624,Lower middle income,1
1,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,2,Civilians (Algeria),7,27,Civilian targeting,Algeria,Hassasna,36.133,0.883,Two citizens were beheaded in Hassasna.,2,0.624,Lower middle income,1
2,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,2,Civilians (Algeria),7,27,Civilian targeting,Algeria,Hassi El Abed,34.966,-0.290,Two citizens were killed in a raid on the vill...,2,0.624,Lower middle income,1


__Теперь, когда мы добавили новые признаки, мы хотим взять случайную подвыборку из нашего общего датасета, чтобы уже на ней обучать модели__


In [430]:
np.random.seed(323)
indexe_vib = np.random.choice(np.arange(len(df)), size = 1*10**4)
df_vib = df.iloc[indexe_vib]


___Проверим, что наша подвыборка вообще будет репрезентативной___, то есть, что она принадлежит к тому же распределению, что и основная. Делать мы это буддем с помощью теста Колмогорова-Смирнова.


In [431]:
test = kstest(df_vib['index'], df['index'])
test

KstestResult(statistic=0.004415135016223926, pvalue=0.9897643357268266)

p-value получился 0.98, а значит, все хорошо и выборка репрезентативна

Однако теперь у нас появилась функциональная зависимость между признаками INTERACTION, SUB_EVENT_TYPE, FATALITIES и violence_ind, что может плохо сказаться на обучении моделей, уберем, например INTERACTION.

In [432]:
df_vib = df_vib.drop('SUB_EVENT_TYPE', axis = 1)

In [433]:
df_vib.to_csv('df_vib.csv', encoding='utf-8', index=False)

> ### Теперь, когда новые признаки созданы, можем приступать к обучению моделей